In [1]:
import pandas as pd

In [8]:
def get_per_thread_timings(
    run_output_file_path,
    statistics=("mean", "std", "min", "max"),
    operations=("Up. determin.", "Up. stochastic", "Weight")
):
    table_rule_line_numbers = []
    with open(run_output_file_path, "r") as output_file:
        for line_number, line in enumerate(output_file):
            if line.strip(" ").startswith("─" * 50):
                table_rule_line_numbers.append(line_number)
    number_of_ranks = len(table_rule_line_numbers) // 3
    timing_dataframes = {}
    for rank in range(number_of_ranks):
        _, header_rule_line, end_rule_line = table_rule_line_numbers[rank * 3:(rank + 1) * 3]
        timing_dataframes[f"Rank {rank}"] = pd.read_fwf(
            run_output_file_path,
            skiprows=(
                list(range(0, header_rule_line - 1)) 
                + [header_rule_line] 
                + list(range(end_rule_line , line_number + 1))
            ),
            header=0
        )
    summary_stats = {key: [] for key in ["rank", "operation"] + list(statistics)}
    for rank in range(number_of_ranks):
        dataframe = timing_dataframes[f"Rank {rank}"]
        for operation in operations:
            timings = pd.to_timedelta(
                dataframe[dataframe.Section.str.strip().str.startswith(operation)].time
            )
            summary_stats["rank"].append(rank)
            summary_stats["operation"].append(operation)
            for stat in statistics:
                summary_stats[stat].append(getattr(timings, stat)().total_seconds())
    summary_stats = pd.DataFrame(summary_stats)
    return timing_dataframes, summary_stats

In [10]:
run_output_file_path = "ParticleDAScaling160Longer1BLASThread.o395270"
timing_dataframes, summary_stats = get_per_thread_timings(run_output_file_path)
print(summary_stats)

    rank       operation      mean       std      min     max
0      0   Up. determin.  1.202200  0.213851  0.93000  2.0200
1      0  Up. stochastic  1.046350  0.144915  0.84700  1.4900
2      0          Weight  0.006631  0.008476  0.00203  0.0416
3      1   Up. determin.  1.318625  0.776672  0.90900  4.0500
4      1  Up. stochastic  1.044950  0.181585  0.75400  1.7900
5      1          Weight  0.004057  0.005337  0.00203  0.0289
6      2   Up. determin.  1.320475  0.276602  0.92600  1.7900
7      2  Up. stochastic  1.160150  0.339650  0.74000  2.4000
8      2          Weight  0.005271  0.012202  0.00203  0.0751
9      3   Up. determin.  1.562925  0.787751  0.97700  3.8300
10     3  Up. stochastic  1.090925  0.319656  0.72500  2.8200
11     3          Weight  0.007265  0.008803  0.00211  0.0425
